In [1]:
dataPath = "./"

Brent <- read.table(paste(dataPath,'BRENT.csv',sep = '/'), header=TRUE,sep=';')
Copper <- read.table(paste(dataPath,'COPPER.csv',sep = '/'), header=TRUE,sep=';')
EurUsd <- read.table(paste(dataPath,'EURUSD.csv',sep = '/'), header=TRUE,sep=';')
Gold <- read.table(paste(dataPath,'GOLD.csv',sep = '/'), header=TRUE,sep=';')
Sber <- read.table(paste(dataPath,'SBER.csv',sep = '/'), header=TRUE,sep=';')
Spx <- read.table(paste(dataPath,'SPX.csv',sep = '/'), header=TRUE,sep=';')
UsdRub <- read.table(paste(dataPath,'USDRUB.csv',sep = '/'), header=TRUE,sep=';')

In [2]:
head(Sber)

,X.DATE.,X.CLOSE.
,<chr>,<dbl>
1,24.03.2022,136.24
2,25.03.2022,131.50
3,28.03.2022,125.00
4,29.03.2022,128.77
5,30.03.2022,134.60
6,31.03.2022,143.69


In [3]:
Brent <- Brent[!duplicated(Brent$X.DATE.), ]
Copper <- Copper[!duplicated(Copper$X.DATE.), ]
EurUsd <- EurUsd[!duplicated(EurUsd$X.DATE.), ]
Gold <- Gold[!duplicated(Gold$X.DATE.), ]
Sber <- Sber[!duplicated(Sber$X.DATE.), ]
Spx <- Spx[!duplicated(Spx$X.DATE.), ]
UsdRub <- UsdRub[!duplicated(UsdRub$X.DATE.), ]

In [4]:
names(Brent)[2] <- "Brent"
names(Copper)[2] <- "Copper"
names(EurUsd)[2] <- "EurUsd"
names(Gold)[2] <- "Gold"
names(Sber)[2] <- "Sber"
names(Spx)[2] <- "Spx"
names(UsdRub)[2] <- "UsdRub"

In [5]:
Stocks <- merge(Sber, Spx, by="X.DATE.",all.x=TRUE, all.y = TRUE)
Stocks <- merge(Stocks, Gold, by="X.DATE.",all.x=TRUE, all.y = TRUE)
Stocks <- merge(Stocks, Brent, by="X.DATE.",all.x=TRUE, all.y = TRUE)
Stocks <- merge(Stocks, Copper, by="X.DATE.",all.x=TRUE, all.y = TRUE)
Stocks <- merge(Stocks, EurUsd, by="X.DATE.",all.x=TRUE, all.y = TRUE)
Stocks <- merge(Stocks, UsdRub, by="X.DATE.",all.x=TRUE, all.y = TRUE)

In [6]:
Stocks <- Stocks[complete.cases(Stocks), ]

In [7]:
install.packages("xts")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘zoo’




In [8]:
library(xts)

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [11]:
rates <- as.xts(as.data.frame(lapply(as.data.frame(lapply(Stocks[2:8], log)), diff)),
                    order.by = as.Date(Stocks$X.DATE.[2:dim(Stocks)[1]],"%d.%m.%Y"))

In [16]:
Stocks <- as.data.frame(rates)

In [18]:
head(Stocks)

,Sber,Spx,Gold,Brent,Copper,EurUsd,UsdRub
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2022-03-24,-0.1872760,0.12977122,0.07597249,0.3553464,0.1781605,0.04312101,0.29615182
2022-03-25,-0.7297687,-0.07444186,-0.03182372,0.3762252,0.1985767,0.01250732,0.12870805
2022-03-29,-0.8193764,-0.09560652,-0.06580527,0.2803399,0.2221870,0.02615274,-0.07104707
2022-03-30,-0.7168971,-0.06772941,-0.06189821,0.2937062,0.2064143,0.02882547,-0.07121164
2022-03-31,-0.6527427,-0.06726895,-0.05987660,0.2695171,0.2057332,0.02454812,-0.09624089
2022-04-01,-0.6480686,-0.12226790,-0.08209481,0.2268497,0.2091571,0.01921115,-0.07562027


In [21]:
regr <- lm(Stocks$Sber ~ Stocks$Spx+Stocks$Gold+Stocks$Brent+Stocks$Copper+Stocks$EurUsd+Stocks$UsdRub, data=Stocks)
summary(regr)


Call:
lm(formula = Stocks$Sber ~ Stocks$Spx + Stocks$Gold + Stocks$Brent + 
    Stocks$Copper + Stocks$EurUsd + Stocks$UsdRub, data = Stocks)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.33915 -0.07401 -0.00601  0.07386  0.33439 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    0.0003836  0.0049142   0.078 0.937810    
Stocks$Spx     0.2941338  0.0793687   3.706 0.000236 ***
Stocks$Gold    1.9711016  0.1600643  12.314  < 2e-16 ***
Stocks$Brent  -0.1613032  0.0383906  -4.202 3.18e-05 ***
Stocks$Copper -1.8904706  0.0870448 -21.718  < 2e-16 ***
Stocks$EurUsd  2.8382421  0.3035298   9.351  < 2e-16 ***
Stocks$UsdRub  0.7319461  0.0433661  16.878  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1068 on 465 degrees of freedom
Multiple R-squared:  0.9704,	Adjusted R-squared:   0.97 
F-statistic:  2538 on 6 and 465 DF,  p-value: < 2.2e-16
